In [1]:
import pandas as pd
import pyarrow.parquet as pq

# Transformación de Datos

## Transformar y desanidar datos para optimizar y minimizar tablas

### Archivo users_items.parquet

In [2]:
# Direccion del archivo comprimido y reconvertido
users_items_content= 'data/01_users_items.parquet'

# Cargar directamente el archivo JSON comprimido en un DataFrame
users_items_dataset = pq.read_table(users_items_content).to_pandas()
# users_items_dataset = pd.read_parquet(users_items_content)

Constatamos tipos de datos y forma de la tabla

In [3]:
print(users_items_dataset.dtypes)

user_id        object
items_count     int64
steam_id       object
user_url       object
items          object
dtype: object


In [4]:
print(f"El DataFrame tiene {users_items_dataset.shape[0]} filas y {users_items_dataset.shape[1]} columnas.")

El DataFrame tiene 88310 filas y 5 columnas.


In [5]:
users_items_dataset.head(2)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."


### Desanidado de la columna 'items'

Como pudimos apreciar, se puede ver que items debe expandirse. En este caso podemos inferir que la columna items contiene una lista, por lo que se puede hacer un codigo diferente al de user_reviews.

In [6]:
# Creamos una lista para almacenar los datos desanidados
dataset_desanidado = []

# Iterar sobre cada fila del DataFrame original
for index, row in users_items_dataset.iterrows():
    user_id = row['user_id']
    items_count = row['items_count']
    steam_id = row['steam_id']
    user_url = row['user_url']

    # Iterar sobre cada elemento en la lista 'items' de la fila actual
    for item in row['items']:
        new_row = {
            'user_id': user_id,
            'steam_id': steam_id,
            'user_url': user_url,
            'item_id': item['item_id'],
            'item_name': item['item_name'],
            'playtime_forever': item['playtime_forever'],
            'playtime_2weeks': item['playtime_2weeks']
        }

        dataset_desanidado.append(new_row)

# Crear un nuevo DataFrame a partir de la lista desanidada
users_items_dataset_desanidado = pd.DataFrame(dataset_desanidado)


In [7]:
print(users_items_dataset.dtypes)

user_id        object
items_count     int64
steam_id       object
user_url       object
items          object
dtype: object


In [8]:
users_items_dataset_desanidado.head(2)

,user_id,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0


In [9]:
print(f"El DataFrame tiene {users_items_dataset_desanidado.shape[0]} filas y {users_items_dataset_desanidado.shape[1]} columnas.")

El DataFrame tiene 5153209 filas y 7 columnas.


### Reduccion y organiazcion del dataframe

Para comenzar se pueden revisar algunas columnas que a simple vista parecen poco útiles para el desarrollo de nuestro trabajo.

#### Columna 'playtime_2weeks'

La columna playtime_2weeks parece que no aporta valor aplicable a los pedidos del trabajo y por los que se ve en las primeras filas, puede no contener gran cantidad de datos.

In [10]:
# Contar valores distintos de cero en la columna 'playtime_2weeks'
valores_distintos_de_cero = (users_items_dataset_desanidado['playtime_2weeks'] != 0).sum()
# Calcular el porcentaje
total_valores = len(users_items_dataset_desanidado['playtime_2weeks'])
porcentaje_valores_distintos_de_cero = (valores_distintos_de_cero / total_valores) * 100

print(f"Porcentaje de valores distintos de cero en 'playtime_2weeks': {porcentaje_valores_distintos_de_cero:.2f}%")

Porcentaje de valores distintos de cero en 'playtime_2weeks': 2.69%


Claramente es un valor totalmente insignificante, por lo tanto la eliminamos.

In [11]:
# Eliminamos columna 'playtime_2weeks'
try:
    users_items_dataset_desanidado = users_items_dataset_desanidado.drop('playtime_2weeks', axis=1)
except:
    print("La columna ya fue eliminada o no existe")

#### Columnas 'steam_id' y 'user_id'

Como estas dos columnas son id's, podríamos pensar que tienen similitudes, de manera que ver si ambas son parecidas, ayudaría a limpiar la tabla, por lo tanto una no sería necesaria. Además en los pedidos para los endpoints de recomendaciones se debe ingresar un id de usuario, que es el que se comparte con el df user_reviews, por lo que 'stem_id' no sería necesario.

In [12]:
# Preparamos las 2 columnas a comparar
columna_steam_id = users_items_dataset_desanidado['steam_id']
columna_user_id = users_items_dataset_desanidado['user_id']

# Calcular el número de valores coincidentes entre las columnas
valores_coincidentes = (columna_steam_id == columna_user_id).sum()

# Calcular el total de elementos en las columnas
total_elementos = len(columna_steam_id)

# Calcular el porcentaje de similitud
porcentaje_similitud = (valores_coincidentes / total_elementos) * 100

print(f"Porcentaje de similitud entre 'steam_id' y 'user_id': {porcentaje_similitud:.2f}%")


Porcentaje de similitud entre 'steam_id' y 'user_id': 42.74%


Evidentemente poseen datos diferentes, por lo que podemos continuar investigando.

In [13]:
# Filtrar las filas donde 'steam_id' no es un número
filas_no_numericas = users_items_dataset_desanidado[users_items_dataset_desanidado['steam_id'].apply(lambda x: not x.isnumeric())]

print("Filas donde 'steam_id' no es numérica:")
print(filas_no_numericas['steam_id'])

Filas donde 'steam_id' no es numérica:
Series([], Name: steam_id, dtype: object)


De aquí podemos inferir que 'steam_id' solo posee numeros en su interior y a continuación podemos corroborarlo definitivamente.

In [14]:
# Forzar cambio de tipo de dato int
try:
    users_items_dataset_desanidado['steam_id'] = users_items_dataset_desanidado['steam_id'].astype(int)
except Exception as e:
    print(f"Ocurrió un error, 'steam_id' no posee unicamente valores numericos o no existe.")
print(users_items_dataset_desanidado.dtypes)

user_id             object
steam_id             int64
user_url            object
item_id             object
item_name           object
playtime_forever     int64
dtype: object


Esto indica claramente que 'steam_id' posee solo numeros

Ahora resta ver si user_id es solo numerica o no. Ya se sabe que no es igual a 'steam_id', que es completmente numérico.

In [15]:
# Filtrar las filas donde 'user_id' es una cadena de texto
filas_no_numericas = users_items_dataset_desanidado[users_items_dataset_desanidado['user_id'].apply(lambda x: not x.isnumeric())]

# Mostrar la información de las filas donde 'user_id' es una cadena de texto
print("Filas donde 'user_id' no es numérica:")
print(filas_no_numericas['user_id'])

Filas donde 'user_id' no es numérica:
277          js41637
278          js41637
279          js41637
280          js41637
281          js41637
             ...    
5152652     POMFP0MF
5152999    ArkPlays7
5153000    ArkPlays7
5153001    ArkPlays7
5153002    ArkPlays7
Name: user_id, Length: 2900676, dtype: object


Aqui vemos claramente que no son solo numeros y además que los user_id de este dataframe son iguales a los del df 'users_items'. Esto deja claro que se puede borrar 'steam_id'.

In [16]:
# Eliminamos columna 'steam_id'
try:
    users_items_dataset_desanidado = users_items_dataset_desanidado.drop('steam_id', axis=1)
except:
    print("La columna ya fue eliminada o no existe")

Se puede decir que para todas las corroboraciones anteriores, se podía hacer una conversion forzosa a int y la columna que no quedara, iba a ser mas claramente la que debiamos dejar porque se correspondería a el 'users_id' del dataframe 'user_reviews'.

#### Columna 'item_id'

Aqui podemos ver si 'item_id' es solo numerica haciendo una conversion directa y si funciona era numérica, de lo contrario es otro tipo de dato y no será cambiado.

In [17]:
# Forzar cambio de tipo de dato int
try:
    users_items_dataset_desanidado['item_id'] = users_items_dataset_desanidado['item_id'].astype(int)
except Exception as e:
    print(f"Ocurrió un error, 'item_id' no posee unicamente valores numericos o no existe.")
print(users_items_dataset_desanidado.dtypes)

user_id             object
user_url            object
item_id              int64
item_name           object
playtime_forever     int64
dtype: object


#### Columna 'playtime_forever'

Aqui tambien podemos ver si 'playtime_forever' es solo numerica, haciendo una conversion forzada.

In [18]:
# Forzar cambio de tipo de dato int
try:
    users_items_dataset_desanidado['playtime_forever'] = users_items_dataset_desanidado['playtime_forever'].astype(int)
except Exception as e:
    print(f"Ocurrió un error, 'item_id' no posee unicamente valores numericos o no existe.")
print(users_items_dataset_desanidado.dtypes)

user_id             object
user_url            object
item_id              int64
item_name           object
playtime_forever     int64
dtype: object


#### Columna 'user_url'

Como en anteriores dataframes, se eliminará 'user_url' para mejorar posterior trabajo en el df y dejarlo mas liviano.

In [19]:
# Eliminamos columna 'user_url'
try:
    users_items_dataset_desanidado = users_items_dataset_desanidado.drop('user_url', axis=1)
except:
    print("La columna ya fue eliminada o no existe")

### Reordenamiento y revision final del dataframe

In [20]:
# Reordenamiento de la posición de las colummnas para mejor uso
reordenamiento_columnas = ['user_id', 'item_name','item_id', 'playtime_forever']
users_items_dataset_desanidado = users_items_dataset_desanidado[reordenamiento_columnas]


In [21]:
# Verificar si hay columnas faltantes o adicionales
columnas_df = set(users_items_dataset_desanidado.columns)
columnas_reordenadas = set(reordenamiento_columnas)

columnas_faltantes = columnas_df - columnas_reordenadas
columnas_adicionales = columnas_reordenadas - columnas_df

if columnas_faltantes:
    print(f'Columnas faltantes: {columnas_faltantes}')
elif columnas_adicionales:
    print(f'Columnas adicionales: {columnas_adicionales}')
else:
    print('Se están utilizando todas las columnas correctamente.')

Se están utilizando todas las columnas correctamente.


In [22]:
users_items_dataset_desanidado.head(2)

,user_id,item_name,item_id,playtime_forever
0,76561197970982479,Counter-Strike,10,6
1,76561197970982479,Team Fortress Classic,20,0


In [23]:
print(f"El DataFrame tiene {users_items_dataset_desanidado.shape[0]} filas y {users_items_dataset_desanidado.shape[1]} columnas.")

El DataFrame tiene 5153209 filas y 4 columnas.


In [24]:
print(users_items_dataset_desanidado.dtypes)

user_id             object
item_name           object
item_id              int64
playtime_forever     int64
dtype: object


In [25]:
# Guardar DataFrame en un archivo Parquet
users_items_dataset_desanidado.to_parquet('data/02_users_items_final.parquet', index=False)